가수 데이터셋을 활용해서 리스트 뽑아내기


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


가수정보 데이터셋 확인

In [46]:
import pandas as pd

# Load the data
file_path = '/content/drive/MyDrive/1104artist.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()

,Unnamed: 0,name,nation,gender,group,era,genre
0,0,뉴진스,KO,f,y,2020,"댄스, R&B/Soul, 국내드라마"
1,1,다비치,KO,f,y,"2000,2010,2020","발라드, 국내드라마, 댄스"
2,2,블랙핑크,KO,f,y,"2010,2020","댄스, 랩/힙합, POP"
3,3,스테이씨,KO,f,y,2020,"댄스, R&B/Soul, 일렉트로니카"
4,4,프로미스나인,KO,f,y,"2010,2020","댄스, 발라드, 국내드라마"


In [47]:
artist_list = data['name'].drop_duplicates().tolist()
artist_list

['뉴진스',
 '다비치',
 '블랙핑크',
 '스테이씨',
 '프로미스나인',
 '레드벨벳',
 'NCT127',
 '방탄소년단',
 '세븐틴',
 '데이식스',
 '몬스타엑스',
 '루시',
 '아이유',
 '이영지',
 '백예린',
 '볼빨간사춘기',
 '태연',
 '정국',
 '성시경',
 '박재범',
 '쿠기',
 '임한별',
 '새소년',
 'wavetoearth',
 '잔나비',
 '검정치마',
 '쏜애플',
 '10cm',
 '우효',
 '최유리',
 '치즈',
 '크러쉬',
 '기리보이',
 '아리아나그란데',
 '테일러스위프트',
 '켈리클락슨',
 '도자캣',
 '휘트니휴스턴',
 '켈라니',
 '에드시런',
 '맥밀러',
 '콜드플레이',
 '마이클잭슨',
 '찰리푸스',
 '더위캔드',
 '지바노프',
 '제이미',
 'ELO',
 '키디비',
 '김범수',
 '릴러말즈',
 '주영',
 '이하이',
 'GSoul',
 'Rad Museum',
 '이바다',
 '홍다빈',
 '빈센트블루',
 'DeVita',
 '딘',
 '레오',
 '박봄',
 '헤이즈',
 '콜드',
 '미소',
 'Chancellor',
 '정엽',
 '거미',
 'SOMA',
 '정기고',
 '휘성',
 '베이빌론',
 '보니',
 'Gist',
 '구원찬',
 'Ra.D',
 '밤비로얄프린스',
 '브라운아이드소울',
 '범키',
 'Otis lim',
 '비비',
 '대니딕',
 '빅나티',
 'DUT2',
 '서사무엘',
 '문수진',
 '원슈타인',
 '다운',
 '자이언티',
 '수비',
 'SURAN',
 '나얼',
 '시도',
 '키엘',
 '유라',
 '루디',
 'SOLE',
 '브라운아이즈',
 '브라운 아이즈',
 '로이킴',
 '케이시',
 'MC the max',
 '이무진',
 '데이먼스이어',
 '산들',
 '우디',
 '서인국',
 'SG워너비',
 '장덕철',
 '임영웅',
 '경서',
 '김필',
 '디셈버',
 '김민석',
 '바

## 1. crawling


In [48]:
!pip install selenium

In [49]:
!pip install beautifulsoup4

In [50]:
from selenium.webdriver.common.by import By

In [51]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat

In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time


# options.add_argument('headless')  # 배경 실행
df_music = pd.DataFrame()

def search_artist_songs(artist_name):
    global df_music
    # 크롬드라이버 열기
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()  # 크롬창 크기 최대

        # 드라이버가 해당 url 접속
    url = 'https://www.melon.com/index.htm'  # 멜론 메인 페이지
    driver.get(url)
    time.sleep(2)

    # 검색창 클릭
    element = driver.find_element(By.XPATH, '//*[@id="top_search"]')
    element.click()
    element.send_keys(artist_name)
    search_element = driver.find_element(By.XPATH, '//*[@id="gnb"]/fieldset/button[2]/span').click()
    time.sleep(2)

    song_element = driver.find_element(By.XPATH, '//*[@id="conts"]/div[4]/div/div[1]/a/span[1]')
    song_element.click()
    time.sleep(2)

    # html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 노래 정보 가져오기
    # 곡 정보 div class
    song_containers = soup.find_all('div', class_='wrap pd_none')
    song_list = []
    # 각 컨테이너에 대해 처리
    for container in song_containers:
        # 먼저 <a> 태그로 시도
        a_tag = container.find('a', class_='fc_gray')
        if a_tag:  # <a> 태그가 존재하면 그 값을 사용
            song_list.append(a_tag.get_text().strip())
        else:  # <a> 태그가 없으면 <span> 태그로 시도
            span_tag = container.find('span', title=True)
            if span_tag:  # <span> 태그가 존재하면 그 값을 사용
                song_list.append(span_tag.get('title').strip())

    # 좋아요 수 가져오기
    like_num_list = [title.get_text() for title in soup.find_all('span', attrs={'class': 'cnt'})]
    # 데이터 정제 후 int 형태로 변환
    like_list = [int(item.replace('\n총건수\n', '').replace(',', '')) for item in like_num_list]
    # 앨범 커버명 가져오기
    album_list = [div.find('a').get_text() for div in soup.find_all('div', class_='ellipsis', style='max-width:90%') if div.find('a')]
    # 아티스트명 가져오기
    artist_list = []

    for div in soup.find_all('div', class_='ellipsis', style='max-width:122px'):
        artists = div.find_all('a', class_='fc_mgray')  # Find all artist <a> tags within the div
        unique_artists = set()  # Create a set to keep track of unique artist names
        for artist in artists:
            artist_name = artist.get_text().strip()
            unique_artists.add(artist_name)
        artist_names = ', '.join(unique_artists)
        artist_list.append(artist_names)

    df = pd.DataFrame({
            "아티스트": artist_list,
            "좋아요": like_list,
            "곡제목": song_list,
            "앨범명": album_list
        })

    df_music = pd.concat([df_music, df], ignore_index=True)
    print(df_music.tail())
    print("last")
    time.sleep(10)


In [53]:
# 가수 정보 기반으로 곡 정보 데이터셋 df화
for i in range(0, len(artist_list)):
    try:
        search_artist_songs(artist_list[i])
    except Exception as e:
        print(f"An error occurred with {artist_list[i]}: {e}")  # 오류 메시지를 출력하거나 로깅할 수 있습니다.
        continue  # 다음 반복으로 넘어갑니다.


               아티스트     좋아요                 곡제목                          앨범명
11         NewJeans   12852             아름다운 구속                너의 시간 속으로 OST
12         NewJeans  101378                Hurt  NewJeans 1st EP 'New Jeans'
13         NewJeans   32065              Get Up     NewJeans 2nd EP 'Get Up'
14         NewJeans   50090                Zero                         Zero
15  J.I.D, NewJeans    7762  Zero (J.I.D Remix)           Zero (J.I.D Remix)
last
         아티스트    좋아요                                곡제목  \
61        다비치  12684                          슬픈 사랑의 노래   
62  이해리 (다비치)  19049  사랑은 언제나 목마르다 (N번째 연애 X 이해리 (다비치))   
63  이해리 (다비치)   8853     이 소설의 끝을 다시 써보려 해 (장화신고 노래할고양)   
64        다비치  32839                            남자도 우나요   
65  이해리 (다비치)  12056                소주 한 잔 (장화신고 노래할고양)   

                                  앨범명  
61                           AMARANTH  
62  사랑은 언제나 목마르다 (N번째 연애 X 이해리 (다비치))  
63                          복면가왕 102회  
64                

In [54]:
df_music

,아티스트,좋아요,곡제목,앨범명
0,NewJeans,151523,Super Shy,NewJeans 2nd EP 'Get Up'
1,NewJeans,108554,ETA,NewJeans 2nd EP 'Get Up'
2,NewJeans,281475,Hype Boy,NewJeans 1st EP 'New Jeans'
3,"League of Legends, NewJeans",15123,GODS,2023 리그 오브 레전드 월드 챔피언십 주제곡
4,NewJeans,271952,Ditto,NewJeans 'OMG'
...,...,...,...,...
10847,9와 숫자들,1056,평정심,"9와 숫자들 99%, 베스트"
10848,9와 숫자들,378,겨울 독수리,보물섬
10849,유정목 (9와 숫자들),111,신록의 계절,신록의 계절
10850,유정목 (9와 숫자들),130,"그때로, 그대에게로",춘분 Vol.2


In [56]:
df_data = df_music[df_music['좋아요'] >= 50]

In [55]:
df_music['아티스트'].unique()

array(['NewJeans', 'League of Legends, NewJeans', 'J.I.D, NewJeans', ...,
       '9 (9와 숫자들)', 'BOY.D (보이.디 of 9와 숫자들)', '유정목 (9와 숫자들)'],
      dtype=object)

In [57]:
df_data

,아티스트,좋아요,곡제목,앨범명
0,NewJeans,151523,Super Shy,NewJeans 2nd EP 'Get Up'
1,NewJeans,108554,ETA,NewJeans 2nd EP 'Get Up'
2,NewJeans,281475,Hype Boy,NewJeans 1st EP 'New Jeans'
3,"League of Legends, NewJeans",15123,GODS,2023 리그 오브 레전드 월드 챔피언십 주제곡
4,NewJeans,271952,Ditto,NewJeans 'OMG'
...,...,...,...,...
10847,9와 숫자들,1056,평정심,"9와 숫자들 99%, 베스트"
10848,9와 숫자들,378,겨울 독수리,보물섬
10849,유정목 (9와 숫자들),111,신록의 계절,신록의 계절
10850,유정목 (9와 숫자들),130,"그때로, 그대에게로",춘분 Vol.2


In [58]:
df_data['아티스트'].unique()

array(['NewJeans', 'League of Legends, NewJeans', 'J.I.D, NewJeans',
       '다비치', '잔나비 최정훈, 강민경 (다비치)', '티아라, 다비치, 씨야', '티아라, 다비치',
       '강민경 (다비치), 박지헌', '블랙펄, 다비치, 씨야', '강민경 (다비치)', '매드클라운, 이해리 (다비치)',
       '이해리 (다비치)', '2BIC(투빅), 다비치', 'BLACKPINK', 'Dua Lipa, BLACKPINK',
       'BLACKPINK, Lady Gaga', 'STAYC(스테이씨)', '프로미스나인', '박지원 (프로미스나인)',
       '송하영 (프로미스나인)', '이나경 (프로미스나인), 박지원 (프로미스나인)', '정동원, 박지원 (프로미스나인)',
       'Red Velvet (레드벨벳)', 'aespa, Red Velvet (레드벨벳)', '레드벨벳-아이린&슬기',
       'NCT 127', '방탄소년단', '정국, 방탄소년단, Charlie Puth', '방탄소년단, 정국',
       '방탄소년단, Coldplay', '방탄소년단, Jason Derulo, Jawsh 685',
       '세븐틴 (SEVENTEEN)', 'DAY6 (데이식스)', '기현 (몬스타엑스)',
       '소유 (SOYOU), 유승우, 보이프렌드, 몬스타엑스, 우주소녀, 정세운, 브라더수, 케이윌, 듀에토, 마인드유',
       '몬스타엑스', '셔누X형원 (몬스타엑스)', '셔누 (몬스타엑스), 민혁 (몬스타엑스)',
       '씨스타, 유승우, 보이프렌드, 몬스타엑스, 정기고, 엑시 (우주소녀), 매드클라운, 주영 (Jooyoung), 브라더수, 케이윌',
       '주헌 (몬스타엑스)',
       '소유 (SOYOU), 유승우, 보이프렌드, 몬스타엑스, 정기고, 매드클라운, 우주소녀, #Gun, 브라더수, 정세운, 케이윌, 이광현, 듀에

In [59]:
df_data.to_csv('/content/drive/MyDrive/1105_songs.csv')